# Crustal Thickness Calculation using Airy Isostasy

This notebook processes paleotopography NetCDF files to compute crustal thickness using the Airy isostasy model. It reads elevation data, applies the isostasy formula, and saves the results in new NetCDF files.


In [ ]:
import xarray as xr
import os
import re

## Define Input and Output Directories

Specify the directories for input NetCDF files and output processed files.


In [ ]:
input_dir = "STELLAR-Phase4A-Paleotopography-Merged-PMAG"
output_dir = "CrustalThickness_P4a_airy_PMAG"
os.makedirs(output_dir, exist_ok=True)
print("Input dir:", input_dir)
print("Output dir:", output_dir)

## Constants

Define constants used in the Airy isostasy calculation.


In [ ]:
RHO_M = 3300  # Mantle density in kg/m³
RHO_C = 2700  # Crust density in kg/m³
H0 = 35000    # Reference crustal thickness in meters

## Conversion Function

Function to convert elevation to crustal thickness using the Airy isostasy model.


In [ ]:
def convert_elevation_to_thickness(elevation):
    """Convert elevation (m) to crustal thickness (m) using Airy isostasy."""
    return H0 + (elevation * RHO_M) / (RHO_M - RHO_C)

## NetCDF Processing Function

Function to load a NetCDF file, compute crustal thickness, and save the result.


In [ ]:
def process_netcdf_file(filename):
    """Load NetCDF, convert elevation to crustal thickness, and save new NetCDF."""
    ds = xr.open_dataset(filename)
    elevation = ds["z"]
    crust_thickness = convert_elevation_to_thickness(elevation)

    thickness_ds = xr.Dataset(
        {"z": (elevation.dims, crust_thickness.data)},
        coords=elevation.coords,
        attrs={"description": "Crustal thickness derived from paleoelevation using Airy isostasy"}
    )

    thickness_ds["z"].attrs = {
        "units": "m",
        "long_name": "Crustal Thickness"
    }

    age = re.search(r"(\d+)Ma", filename).group(1)
    out_filename = os.path.join(output_dir, f"crustal_thickness_{age}Ma.nc")

    encoding = {
        "z": {
            "zlib": True,
            "complevel": 1,
            "shuffle": True
        }
    }

    thickness_ds.to_netcdf(out_filename, encoding=encoding)
    print(f"Saved: {out_filename}")

## Batch Processing

Process all matching NetCDF files in the input directory.


In [ ]:
for file in os.listdir(input_dir):
    if file.startswith("paleotopography_spliced_") and file.endswith(".nc"):
        process_netcdf_file(os.path.join(input_dir, file))